In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from tensorflow.keras.applications import VGG16, InceptionV3, ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Resizing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from drive.MyDrive.DL.codes.transcription.utils import plot_training
import tensorflow as tf
from drive.MyDrive.DL.codes.hatespeech.pipelines import ML,binarize,vectorize
import warnings
warnings.filterwarnings('ignore')
obj = 'txt'
early_stopping = EarlyStopping(monitor='loss', patience=5)

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/train',labels='inferred',label_mode='int',color_mode='rgb',batch_size=32,image_size=(299,299))
valid = tf.keras.utils.image_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/val',labels='inferred',label_mode='int',color_mode='rgb',batch_size=32,image_size=(299,299))
test  = tf.keras.utils.image_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/test',labels='inferred',label_mode='int',color_mode='rgb',batch_size=32,image_size=(299,299))

train = train.map(binarize)
valid = valid.map(binarize)
test = test.map(binarize)

inception_model = InceptionV3(include_top=False,input_shape=(299,299,3))
x = Flatten()(inception_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inception_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    train,
    validation_data=valid,
    epochs=150,
    callbacks=[early_stopping]
)

model.evaluate(test)
plot_training(history)

Found 71496 files belonging to 6 classes.
Found 5000 files belonging to 6 classes.
Found 10000 files belonging to 6 classes.
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 4609s 2s/step - accuracy: 0.6276 - loss: 0.8769 - val_accuracy: 0.5510 - val_loss: 0.7048
Epoch 2/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 372s 166ms/step - accuracy: 0.6359 - loss: 0.6574 - val_accuracy: 0.5510 - val_loss: 0.6985
Epoch 3/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 370s 166ms/step - accuracy: 0.6358 - loss: 0.6568 - val_accuracy: 0.5510 - val_loss: 0.6994
Epoch 4/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 370s 165ms/step - accuracy: 0.6360 - loss: 0.6559 - val_accuracy: 0.5510 - val_loss: 0.7009
Epoch 5/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 370s 165ms/step - accuracy: 0.6356 - loss: 0.6581 - val_accuracy: 0.5510 - val_loss: 0.7001
Epoch 6/150
2235/2235 ━━━━━━━━━━━━━━━━━━━━ 364s 163ms/step - accuracy: 0.6362 - loss: 0.6557 - val_accuracy: 0.5510 - val_loss: 0.7005
Epoch 7/150
2235

In [8]:
train = tf.keras.utils.text_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/train',labels='inferred',label_mode='int',batch_size=32)
valid = tf.keras.utils.text_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/val',labels='inferred',label_mode='int',batch_size=32)
test  = tf.keras.utils.text_dataset_from_directory(f'/content/drive/MyDrive/DL/PREG1/hate-speech/{obj}/test',labels='inferred',label_mode='int',batch_size=32)

train = train.map(binarize)
valid = valid.map(binarize)
test = test.map(binarize)

train = train.map(vectorize)
valid = valid.map(vectorize)
test = test.map(vectorize)

inception_model = InceptionV3(include_top=False,input_shape=(299,299,1))

x = Resizing(299, 299,1)(inception_model.input)
x = inception_model(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inception_model.input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    train,
    validation_data=valid,
    epochs=150,
    callbacks=[early_stopping]
)

model.evaluate(test)
plot_training(history)

Found 78886 files belonging to 6 classes.
Found 5000 files belonging to 6 classes.
Found 10000 files belonging to 6 classes.


OperatorNotAllowedInGraphError: in user code:

    File "/content/drive/MyDrive/DL/codes/hatespeech/pipelines.py", line 124, in vectorize  *
        texts = model.fit_transform(texts)
    File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_set_output.py", line 1045, in wrapped  *
        data_to_wrap = f(self, X, *args, **kwargs)
    File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1097, in fit_transform  *
        return self.fit(X, **fit_params).transform(X)
    File "/content/drive/MyDrive/DL/codes/hatespeech/pipelines.py", line 87, in fit  *
        tokenized_sentences = [sentence.split() for sentence in X]

    OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does no